# Alternating least squares for Canonical Polyadic (CP) Decomposition

```
Copyright 2025 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

The function `cp_als` computes an estimate of the best rank-$R$ CP model of a tensor $\mathcal{X}$ using the well-known alternating least-squares algorithm (see, e.g., Kolda and Bader, SIAM Review, 2009, for more information). The input $\mathcal{X}$ can be almost any type of tensor including a `tensor`, `sptensor`, `ktensor`, or `ttensor`. The output CP model is a `ktensor`.

In [ ]:
import os
import sys
import pyttb as ttb
import numpy as np

## Generate low-rank data tensor

In [ ]:
# Choose the rank and shape
R = 2
tensor_shape = (3, 4, 5)

# Set the random seed for reproducibility
np.random.seed(0)

# Create a low-rank dense tensor from a Kruskal tensor (i.e., ktensor)
factor_matrices = [np.random.randn(s, R) for s in tensor_shape]
M_true = ttb.ktensor(factor_matrices)  # true solution
X = M_true.to_tensor()

## Run `cp_als` using default parameters
The `cp_als` method has two required inputs: a data tensor (`X`) and the rank of the CP model (`R`) to compute.

By default, `cp_als` uses a *random* initial guess. At each iteration, it reports the *fit* `f` which is defined for a data tensor `X` and CP model `M` as 
```
f = 1 - ( X.norm()**2 + M.norm()**2 - 2*<X,M> ) / X.norm()
``` 
and is loosely the proportion of the data described by the CP model, i.e., a fit of 1 is perfect.

In [ ]:
# Compute a solution with final ktensor stored in M1
np.random.seed(1)  # Set the random seed for reproducibility
M1, M1_init, M1_info = ttb.cp_als(X, R)

The `cp_als` method returns the following:
1. `M1`: the solution as a `ktensor`. 
2. `M1_init`: the initial guess as a `ktensor` that was used in computing the solution. 
3. `M1_info`: a `dict` containing runtime information with keys:
    * `params`: parameters used by `cp_als`
    * `iters`: number of iterations performed
    * `normresidual`: the norm of the residual `X.norm()**2 + M.norm()**2 - 2*<X,M>`
    * `fit`: the fit `f` described above

In [ ]:
print("M1_info:")
for k, v in M1_info.items():
    print(f"\t{k}: {v}")

## Run `cp_als` using different initial guesses

Different random initial guesses can be generated and used by setting different random seeds (via `numpy`). You can also explicitly set the `init` parameter to `"random"` to make it clear that the default random initialization is being used, although this is not necessary as illustrated above.

In [ ]:
np.random.seed(2)  # Set seed for reproducibility
M2, M2_init, _ = ttb.cp_als(X, R, init="random")  # leaving third output unassigned

A specific `ktensor` can also be used as an initial guess. As an example, using the same initial guess (and all other parameters) as the previous run of `cp_als` gives the exact same solution.

In [ ]:
M2_rerun, _, _ = ttb.cp_als(X, R, init=M2_init)

The `"nvecs"` initialization option computes the initial guess using the eigenvectors of the outer product of the matricized data tensor. This initialization process will require more computation than the default random initialization, but it can often lead to better solutions in fewer iterations.

In [ ]:
M2_nvecs, _, _ = ttb.cp_als(X, R, init="nvecs")
# score_M3 = M3.score(M_true)
# print(f"Score of M2 and M_true: {score_M2[0]}")
# print(f"Score of M3 and M_true: {score_M3[0]}")

## Evaluate and compare the outputs of `cp_als`
Use the `ktensor.score()` method to compare outputs of `cp_als` to the true solution if known. In the examples above, the data tensor was generated from a `ktensor`, `M_true`, which can be used to evaluate solutions computed using `cp_als`. A score of 1 indicates a perfect match.


Note that the `ktensor.score()` method returns a tuple with the score as the first element and other information related to the score computation as the remaining elements. See the `ktensor` documentation for more information about the return values.

In [ ]:
score_M1 = M1.score(M_true)  # not a good solution
score_M2 = M2.score(M_true)  # a better solution
score_M2_nvecs = M2_nvecs.score(M_true)  # an even better solution

print(f"Score of M1 and M_true: {score_M1[0]}")
print(f"Score of M2 and M_true: {score_M2[0]}")
print(f"Score of M2_nvecs and M_true: {score_M2_nvecs[0]}")

When two solutions match, as is the case with `M2` and `M2_rerun`, the score will be 1 (up to floating point error).

In [ ]:
score_M2_rerun = M2.score(M2_rerun)

print(f"Score of M2 and M2_rerun: {score_M2_rerun[0]}")

## Increase the maximum number of iterations
Note that the previous run kicked out at only 10 iterations, before reaching the specified convegence tolerance. Let's increase the maximum number of iterations and try again, using the same initial guess.

In [ ]:
more_iters = 100
M2_better, _, _ = ttb.cp_als(X, R, maxiters=more_iters, init=M_true)

## Changing the output frequency
Using the `printitn` option to change the output frequency.

In [ ]:
M = ttb.cp_als(X, R, printitn=5)

## Suppress all output
Set `printitn` to zero to suppress all output.

In [ ]:
M = ttb.cp_als(X, R, printitn=0)  # No output

## Use initial guess based 
Use the `"nvecs"` option to initialize `cp_als` with the leading mode-$n$ singular vectors of the input tensor. This initialization process will require more computation than the default `"random` initialization in general, but it can lead to better solutions.

## Change the order of the dimensions in CP
Changing the order of the dimensions in which `cp_als` iterates over the input tensor can lead to a different solution.

In [ ]:
M4, _, M4_info = ttb.cp_als(
    X, R, maxiters=few_iters, init="nvecs", printitn=0, dimorder=[2, 1, 0]
)
score_M4 = M4.score(M_true)
print(f"Score of M3 and M_true: {score_M3[0]}")
print(f"Score of M4 and M_true: {score_M4[0]}")

In the last example, we also collected the third output argument `M4_info` which has runtime information in it. The field `M4_info["iters"]` has the total number of iterations. The field `M4_info["params"]` has the information used to run the method. Unless the initialization method is `"random"`, passing the parameters back to the method will yield the exact same results.

In [ ]:
M4_rerun, _, M4_rerun_info = ttb.cp_als(X, R, init="nvecs", **M4_info["params"])
score_M4_rerun = M4.score(M4_rerun)
print(f"Score of M4 and M4_rerun: {score_M4_rerun[0]}")

print("M4_info:")
for k, v in M4_info.items():
    print(f"\t{k}: {v}")

print("M4_rerun_info:")
for k, v in M4_rerun_info.items():
    print(f"\t{k}: {v}")

## Change the stopping tolerance
It's also possible to loosen or tighten the stopping tolerance on the change in the fit. Note that you may need to increase the number of iterations for it to converge.

In [ ]:
M5 = ttb.cp_als(X, R, init="nvecs", maxiters=1000, stoptol=1e-12, printitn=100)

## Control sign ambiguity of factor matrices
The default behavior of `cp_als` is to make a call to `fixsigns()` to fix the sign ambiguity of the factor matrices. You can turn off this behavior by passing the `fixsigns` parameter value of `False` when calling `cp_als`.

In [ ]:
# Create rank-2 tensor
X2 = ttb.ktensor(
    factor_matrices=[
        np.array([[1.0, 1.0], [-1.0, -10.0]]),
        np.array([[1.0, 1.0], [-2.0, -10.0]]),
    ],
    weights=np.array([1.0, 1.0]),
)
print(f"X2=\n{X2}\n")

M_fixsigns, _, _ = ttb.cp_als(X2, 2, printitn=0, init=ttb.ktensor(X2.factor_matrices))
print(f"M_fixsigns=\n{M_fixsigns}\n")  # default behavior, fixsigns called

M_no_fixsigns, _, _ = ttb.cp_als(
    X2, 2, printitn=0, init=ttb.ktensor(X2.factor_matrices), fixsigns=False
)
print(f"M_no_fixsigns=\n{M_no_fixsigns}\n")  # fixsigns not called

## Recommendations
* Run multiple times with different guesses and select the solution with the best fit.
* Try different ranks and choose the solution that is the best descriptor for your data based on the combination of the fit and the interpretation of the factors, e.g., by visualizing the results.